### Verileri Düzeltmek
- Modelin verileri işleyebilmesi için doğru ve makinenin anlayacağı türden olması lazım
- `NaN` değerler silinmeli ya da doldurulması lazım.
- Veri düzenlemenin iki yolu gösterilmiştir

In [80]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor # Modeler
from sklearn.model_selection import train_test_split  # Veri setinin bölmeye yarar
from sklearn.preprocessing import OneHotEncoder  # Kategorik özellikleri binary özelliklere çevirme
from sklearn.compose import ColumnTransformer    # Sütun bazında dönüşümler

from sklearn.impute import SimpleImputer          # Eksik verileri doldurmak için kullanılır.
from sklearn.compose import ColumnTransformer

model = RandomForestRegressor()

data1 = pd.read_csv("data/car_sales_extended_missing_data.csv")
data2 = pd.read_csv("data/car_sales_extended_missing_data.csv")

# "Price" kısmı sonuç verisi olduğu için bu kısımda olan Nan satırları silinir.
# `inplace` kısmı direkt datanın üstünde işlem yapılması sağlanır.
data1.dropna(subset=["Price"], inplace=True) 
data2.dropna(subset=["Price"], inplace=True) 
# data1.dropna(inplace=True) Nan değeri olan bütün satırlar silinir.

# Data sayısını, data türünü gösterir
len(data1),"_"*50, data1.dtypes, "_"*50, data1.isna().sum()    

(950,
 '__________________________________________________',
 Unnamed: 0         int64
 Make              object
 Colour            object
 Odometer (KM)    float64
 Doors            float64
 Price            float64
 dtype: object,
 '__________________________________________________',
 Unnamed: 0        0
 Make             47
 Colour           46
 Odometer (KM)    48
 Doors            47
 Price             0
 dtype: int64)

---
### 1. Yol

---

In [81]:
# "Doors" sütününda her bir benzersiz değerin kaç kez tekrarlandığını saymak için kullanılır. Kapı sayısında
#  bir ortalama almak mantıklı değil bu yüzden boş olan kısımlara en fazla tekrarlanan veri koyulur
data1["Doors"].value_counts()

Doors
4.0    768
5.0     71
3.0     64
Name: count, dtype: int64

In [82]:
data1["Make"] = data1["Make"].fillna("missing") # Nan değelere "missing" ataması yapılır
data1["Colour"] = data1["Colour"].fillna("missing")

# Nan değerlere o sütünün ortalaması atanır
data1["Odometer (KM)"] = data1["Odometer (KM)"].fillna(data1["Odometer (KM)"].mean()) 
data1["Doors"] = data1["Doors"].fillna(4) # En fazla 4 kapı var
data1.isna().sum()  

Unnamed: 0       0
Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

---

- Model eğitmek için kullanılan verileri bilgisayarın anlayabileceği veriye dönüştürmek gerekir. 
Bu kısımda iki çözüm örneklendirildi. Renkler, markalar bir anlam ifade etmez. 
Bu yüzden 
- `One-hot encoding`, her kategorik değeri bir sütuna dönüştürür ve bu sütunların her birinde ilgili kategoriye ait olan veriler için 1, diğerleri için 0 değeri yer alır.
    - `remainder="passthrough"`: Bu parametre, categorical_features listesine dahil edilmeyen diğer sütunların dönüştürülmeden (orijinal halleriyle) bırakılmasını sağlar. Alternatif olarak, remainder="drop" parametresini kullanarak bu sütunların atılmasını sağlayabilirsiniz.
- `get_dummies` daha anlaşılır bir çıktı verir. Görevi aynıdır fakat `OneHotEncoder` daha iyi sonuç verir

---

In [101]:
categorical_features = ["Make", "Colour", "Doors"]
one_hot = OneHotEncoder()
#
# 
transformer = ColumnTransformer([("one_hot",
                                   one_hot,
                                   categorical_features)],
                                   remainder="passthrough")

tf_X = transformer.fit_transform(data1)

# Ya da get_dummies
dummies = pd.get_dummies(data1[categorical_features])
dummies[:3]

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Make_missing,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White,Colour_missing
0,4.0,False,True,False,False,False,False,False,False,False,True,False
1,5.0,True,False,False,False,False,False,True,False,False,False,False
2,4.0,False,True,False,False,False,False,False,False,False,True,False


In [109]:
y = data1["Price"]
X_train, X_test, y_train, y_test = train_test_split(tf_X, y, test_size=0.2)
model.fit(X_train, y_train)
score1 = model.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(dummies, y, test_size=0.2)
model.fit(X_train, y_train)
score2 = model.score(X_test, y_test)

print(f"Coefficient of Determination: {score1 * 100:.2f} (one_hot)")
print(f"Coefficient of Determination: {score2 * 100:.2f} (dummies)")

Coefficient of Determination: 99.95 (one_hot)
Coefficient of Determination: 12.22 (dummies)


---

### 2. Yol

---

In [110]:
np.random.seed(42)
X = data2.drop("Price", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2)

### Boş Değerlerin Doldurulması
- `SimpleImputer` Eksik değerleri doldurmak için kullanılan sınıftır. `strategy` doldurma sırasında kullanılıcak stratejiyi beliler örnek olarak "mean" (ortalama), "median" (medyan), "most_frequent" (en sık tekrar eden değer) veya "constant" (sabit bir değer) olabilir. `fill_value` eksik değerleri doldurmak için kullanılacak sabit değeri belirtir.
- `ColumnTransformer` Farklı sütunlara farklı dönüşümler veya işlemler uygulamak için kullanılır. İlk parametre dönüşüme ad verme için kullanılır.
- `remainder="passthrough"` parametresi, dönüşüm sırasında diğer sütunların değiştirilmeden kalmasını sağlar.

In [111]:
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
num_imputer = SimpleImputer(strategy="mean")

cat_features = ["Make", "Colour"]
door_feature = ["Doors"]
num_features = ["Odometer (KM)"]

imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_features),
    ("door_imputer", door_imputer, door_feature),
    ("num_imputer", num_imputer, num_features)
])

filled_X_train = imputer.fit_transform(X_train)
filled_X_test = imputer.transform(X_test)

car_sales_filled_train = pd.DataFrame(filled_X_train, 
                                      columns=["Make", "Colour", "Doors", "Odometer (KM)"])

car_sales_filled_test = pd.DataFrame(filled_X_test, 
                                     columns=["Make", "Colour", "Doors", "Odometer (KM)"])
car_sales_filled_test.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [112]:
categorical_features = ["Make", "Colour", "Doors"]
one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot", one_hot, 
                                 categorical_features)],
                                 remainder="passthrough")

transformed_X_train = transformer.fit_transform(car_sales_filled_train)
transformed_X_test = transformer.transform(car_sales_filled_test)
transformed_X_train.toarray()

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 7.19340e+04],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.62665e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 4.28440e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.96225e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.33117e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 1.50582e+05]])

In [113]:
model = RandomForestRegressor()
model.fit(transformed_X_train, y_train)
model.score(transformed_X_test, y_test)

0.21735623151692096